## Launch a container on an edge device - with python-chi



In [1]:
from chi import container, context, lease
import os
import chi

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@Edge")

Change the string in the following cell to reflect the name of *your* lease (**with your own net ID**), then run it to get your lease:

In [2]:
l = lease.get_lease(f"proj_plant_group7_edge_sering") 
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>proj_plant_gro…

Lease Details:
Name: proj_plant_group7_edge_sering
ID: 8af62902-b302-413f-bf61-ba0877463727
Status: ACTIVE
Start Date: 2025-05-10 16:00:00
End Date: 2025-05-13 04:00:00
User ID: 75a6b77d5fa49a3452e731b46cd1bb19ab89434defe5fba57a104b06b20b0621
Project ID: 5f910093b40d4f038aa746c10d270901

Node Reservations:

Floating IP Reservations:

Network Reservations:

Events:


In [3]:
username = os.getenv('USER') # exp resources will have this suffix
c = container.Container(
    name = f"proj_plant_group7_edge_sering".replace('_', '-'),
    reservation_id = l.device_reservations[0]["id"],
    image_ref = "quay.io/jupyter/minimal-notebook:latest", 
    exposed_ports = [8888]
)
c.submit(idempotent=True)

Waiting for container b4d4256d-8e94-4388-8bad-2df73748bfd2 status to turn to Running. This can take a while depending on the image


Property,Value
Name,proj-plant-group7-edge-sering
ID,b4d4256d-8e94-4388-8bad-2df73748bfd2
Status,Running
Image,quay.io/jupyter/minimal-notebook:latest
Created at,None
Exposed Ports,[8888]
Reservation ID,dace895d-9704-4aa4-9815-e715fe25a6c8
Runtime,Default


Then, we’ll associate a floating IP with the container, so that we can access the Jupyter service running in it.

In [4]:
c.associate_floating_ip()

'129.114.34.237'

In the output above, make a note of the floating IP that has been assigned to your container.

Let’s retrieve a copy of these materials on the container:

In [5]:
stdout, code = c.execute("git clone https://github.com/teaching-on-testbeds/serve-edge-chi.git")
print(stdout)

Cloning into 'serve-edge-chi'...



In [6]:
stdout, code = c.execute("mv serve-edge-chi/workspace/models work/")
print(stdout)

In [7]:
stdout, code = c.execute("mv serve-edge-chi/workspace/measure_pi.ipynb work/")
print(stdout)

and, install the ONNX runtime Python module:

In [8]:
stdout, code = c.execute("python3 -m pip install onnxruntime")
print(stdout)

Finally, we will get the container logs. Run:

In [9]:
print(chi.container.get_logs(c.id))

Entered start.sh with args: start-notebook.py
Running hooks in: /usr/local/bin/start-notebook.d as uid: 1000 gid: 100
Done running hooks in: /usr/local/bin/start-notebook.d
Running hooks in: /usr/local/bin/before-notebook.d as uid: 1000 gid: 100
Sourcing shell script: /usr/local/bin/before-notebook.d/10activate-conda-env.sh
Done running hooks in: /usr/local/bin/before-notebook.d
Executing the command: start-notebook.py
Executing: jupyter lab
[I 2025-05-12 02:08:47.824 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-05-12 02:08:47.829 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-05-12 02:08:47.835 ServerApp] jupyterlab | extension was successfully linked.
[I 2025-05-12 02:08:47.839 ServerApp] nbclassic | extension was successfully linked.
[I 2025-05-12 02:08:47.844 ServerApp] notebook | extension was successfully linked.
[I 2025-05-12 02:08:47.845 ServerApp] Writing Jupyter server cookie secret to /home/jovyan/.local/share/jupyter/

and look for a line like

        http://127.0.0.1:8888/lab?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

Paste this into a browser tab, but in place of 127.0.0.1, substitute the floating IP assigned to your container, to open the Jupyter notebook interface that is running *on your Raspberry Pi 5*.

Then, in the file browser on the left side, open the “work” directory and find the `measure_pi.ipynb` notebook to continue.